In [ ]:
#install selenium และ bs4 เพื่อนำมาใช้ในการทำ web scraping

In [29]:
!pip install selenium

In [30]:
!pip install bs4

In [31]:
# import libary ที่จะใช้งาน
from selenium import webdriver
from selenium.webdriver.chrome.service import Service #run web driver
from selenium.webdriver.common.by import By #syntax ที่เลือกใช้จำเป็นต้อง import เจ้าตัวนี้เพื่อใช้ By.XPATH
from selenium.webdriver.common.keys import Keys #ใช้ในการสั่งให้สคริปกด enter
from bs4 import BeautifulSoup as bs #ช่วยในการดึง data ที่สนใจจากหน้าเว็บ
import pandas as pd #แปลง data เป็น data frame 
import time

In [32]:
#เปิด web driver โดย chrome driver version 115 โหลดได้ที่ https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/115.0.5790.102/win32/chromedriver-win32.zip

service = Service(executable_path = r'C:\Users\Mark42\Desktop\shopee\chromedriver-win32\chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [33]:
#สั่ง web driver เข้าไปที่หน้าเว็บหลัก shopee
driver.get('https://shopee.co.th/')

In [34]:
#กดปุ่มเลือกภาษา
choose_thailang = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
choose_thailang.click()

In [35]:
#กดปุ่มปิดโฆษณา
#ติด shadowroot โปรแกรมจะมองไม่เห็น ต้อง execute เขียน java script เพื่อหา
close_ad = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_ad.click()

In [36]:
#กรอกคำค้นหา 
search = driver.find_element(By.XPATH, '/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('โต๊ะทำงาน')

In [37]:
#กด enter
search.send_keys(Keys.ENTER)

In [15]:
#กรอก username สำหรับ login
login_user = driver.find_element(By.NAME, "loginKey")
login_user.send_keys('your username')

In [16]:
#กรอก password สำหรับ login
login_user = driver.find_element(By.NAME, "password")
login_user.send_keys('yourpassword')

In [17]:
#กดปุ่ม login
login_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/button')
login_button.click()

In [ ]:
#Auth when login first time
auth = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[1]/div/div/div/button')
auth.click()

In [19]:
#สร้าง list เตรียมไว้เก็บข้อมูลที่ scraping มา
all_products = []
all_price = []
all_soldAmount = []
all_province = [] 


#หาจำนวนหน้าสินค้าที่แสดงทั้งหมด เพื่อนำมาใช้กับ loop
total_pages = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div[3]/div[1]/div[2]/div/span[2]')
total_pages = total_pages.text #รับค่ามาเป็น text ก่อนจากนั้นแปลงเป็น int เพื่อใช้งาน
total_pages = int(total_pages)

    
for i in range(total_pages): #loop เพื่อเก็บข้อมูลให้ครบทุกหน้า
    
    driver.execute_script("document.body.style.zoom='10%' ") #ซูมออกเพื่อให้หน้าเว็บโหลด products ทั้งหมดในหน้านั้น
    data = driver.page_source #เก็บข้อมูลหน้าเว็บทั้งหมดไว้ในตัวแปร
    soup = bs(data) #ใช้ beautifulsoup เพื่อเรียกข้อมูลเฉพาะส่วนที่สนใจ
    
    
    #เก็บข้อมูลส่วนที่สนใจไว้ใน ตัวแปร โดยระบุผ่าน class
    products = soup.find_all('div', {'class':'ie3A+n bM+7UW Cve6sh'})
    price = soup.find_all('div', {'class': 'vioxXd rVLWG6'})
    sold_amount = soup.find_all('div', {'class': 'ZnrnMl'})
    province = soup.find_all('div', {'class': 'zGGwiV'})
    

    #สร้าง loop เพื่อนำข้อมูลที่อยู่ในตัวแปรลเข้าไปเก็บไว้ใน list ที่สร้างไว้
    for i in products:
        all_products.append(i.text)

    for i in price:
        all_price.append(i.text)

    for i in sold_amount:
        all_soldAmount.append(i.text)

    for i in province:
        all_province.append(i.text)
    
    
    time.sleep(3) #สำหรับคอมที่ช้า... เว้นช่วงเวลาในการรัน
    
    #ปุ่มสำหรับกดไปหน้าถัดไป (ใช้ปุ่มเลื่อนหน้าด้านบน เนื่องจากุ่มทางล่างสุดใช้แล้วมีปัญหา web driver รันหยุดแค่หน้าที่ 3)
    next_page_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/button[2]')
   
    #2 บรรทัดนี้สำหรับแก้ไข error: Element not clickable at point (x, y)
    driver.execute_script("arguments[0].scrollIntoView();", next_page_btn)
    driver.execute_script("arguments[0].click();", next_page_btn)
    
    time.sleep(3)
    

In [20]:
#ตรวจสอบว่ารับข้อมูลมาครบถ้วน

print('Products: ', len(all_products))

print('Price: ', len(all_price))

print('Sold: ', len(all_soldAmount))

print('Provine: ', len(all_province))

Products:  1020
Price:  1020
Sold:  1020
Provine:  1020


In [21]:
all_products

['ALOVE【มี 3 สี】80/100/120cmโต๊ะทํางาน  ราคาถูก สไตล์โมเดิร์นโต๊ะคอมพิวเตอร์  กันน้ำ ง่ายต่อการติดตั้ง รับน้ำหนักได้ 400KG',
 '【มี 3 สี】มั่นคง โต๊ะเรียนขาเหล็กหนา 80/100/120cm',
 'โต๊ะวางโน๊ตบุ๊ค สำหรับเด็ก นักเรียน นักศึกษา วัยทำงาน ใช้ในบ้าน หอพัก 60x40x27.5cm',
 '【Cles】🔥ราคาโปร‼️ โต๊ะคอม 💻โต๊ะวางโน้ตบุ๊ค โต๊ะนั่งกับพื้น โต๊ะเขียนหนังสือ [ขนาด40x60cm.]',
 '[ขาโต๊ะหนา]โต๊ะทํางานสีขาว80/100/120/140CM นักเรียน สไตล์เรียบง่าย  desk',
 'โต๊ะพับ ใช้สำหรับวางโน๊ตบุ๊ค หรือเขียนหนังสือ สำหรับเด็ก นักเรียน นักศึกษา ขนาดเล็ก',
 'yhlovehome โต๊ะญี่ปุ่น  วางโน๊ตบุ๊ค พกพาทำงานนอกสถานที่ได้ YF-1050-2 1050-3D',
 'โต๊ะญี่ปุ่น พับได้ มาพร้อมที่วางแก้วเเละเเท็บเเล็ต โต๊ะพับ มีให้เลือกหลายสี',
 'โต๊ะพับญี่ปุ่น ขนาด 60*40*28 เซ็นติเมตร เหมาะสำหรับทำงาน เรียนหนังสือ อ่านหนังสือ ทานข้าวได้ วางของต่างๆ',
 'DUDEE โต๊ะกาแฟสไตล์ญี่ปุ่น ใช้งานได้หลากหลาย สีสวยเข้ากับคนรักการแต่งห้อง',
 'WoodMall โต๊ะไม้ โต๊ะคอมพิวเตอร์   โต๊ะวางของ Computer Desk โต๊ะคอม หน้าโต๊ะไม้ขนาด 68x60x28cm',
 'AFISH🐟โต๊ะคอมพิวเตอร์มินิมอ

In [22]:
all_price

['฿124 - ฿454',
 '฿124 - ฿454',
 '฿85 - ฿125',
 '฿109',
 '฿494 - ฿1,039',
 '฿79 - ฿109',
 '฿69 - ฿148',
 '฿88',
 '฿79 - ฿159',
 '฿279 - ฿539',
 '฿125',
 '฿119',
 '฿79 - ฿179',
 '฿79 - ฿179',
 '฿49 - ฿103',
 '฿111 - ฿129',
 '฿88 - ฿95',
 '฿129 - ฿139',
 '฿109 - ฿139',
 '฿213 - ฿589',
 '฿239',
 '฿89',
 '฿387 - ฿574',
 '฿99 - ฿459',
 '฿454 - ฿739',
 '฿1',
 '฿86',
 '฿135',
 '฿60 - ฿89',
 '฿109 - ฿139',
 '฿222',
 '฿139',
 '฿59',
 '฿98',
 '฿79 - ฿200',
 '฿129 - ฿139',
 '฿99 - ฿398',
 '฿732 - ฿763',
 '฿69 - ฿175',
 '฿122 - ฿129',
 '฿109',
 '฿124 - ฿454',
 '฿85',
 '฿589',
 '฿199 - ฿469',
 '฿130',
 '฿119',
 '฿264 - ฿939',
 '฿619 - ฿1,239',
 '฿165',
 '฿59',
 '฿299',
 '฿304 - ฿1,459',
 '฿68 - ฿148',
 '฿60 - ฿89',
 '฿294 - ฿699',
 '฿549 - ฿739',
 '฿139',
 '฿224 - ฿939',
 '฿589 - ฿1,459',
 '฿869 - ฿1,249',
 '฿549 - ฿739',
 '฿587 - ฿1,387',
 '฿224 - ฿939',
 '฿264 - ฿666',
 '฿87 - ฿89',
 '฿98',
 '฿224 - ฿1,039',
 '฿200 - ฿445',
 '฿349 - ฿745',
 '฿111 - ฿129',
 '฿145',
 '฿88',
 '฿585 - ฿1,489',
 '฿224

In [23]:
all_soldAmount

['ขายแล้ว 16.9พัน ชิ้น',
 'ขายแล้ว 23พัน ชิ้น',
 'ขายแล้ว 2.2พัน ชิ้น',
 'ขายแล้ว 78 ชิ้น',
 'ขายแล้ว 1.5พัน ชิ้น',
 'ขายแล้ว 97.8พัน ชิ้น',
 'ขายแล้ว 1.3พัน ชิ้น',
 'ขายแล้ว 1พัน ชิ้น',
 'ขายแล้ว 433 ชิ้น',
 'ขายแล้ว 4.2พัน ชิ้น',
 'ขายแล้ว 1.3พัน ชิ้น',
 'ขายแล้ว 34.5พัน ชิ้น',
 'ขายแล้ว 270 ชิ้น',
 'ขายแล้ว 33.7พัน ชิ้น',
 'ขายแล้ว 29.3พัน ชิ้น',
 'ขายแล้ว 5.2พัน ชิ้น',
 'ขายแล้ว 951 ชิ้น',
 'ขายแล้ว 9 ชิ้น',
 'ขายแล้ว 5.4พัน ชิ้น',
 'ขายแล้ว 4.1พัน ชิ้น',
 'ขายแล้ว 41.4พัน ชิ้น',
 'ขายแล้ว 52.4พัน ชิ้น',
 '',
 'ขายแล้ว 15.5พัน ชิ้น',
 'ขายแล้ว 31.7พัน ชิ้น',
 'ขายแล้ว 997.9พัน ชิ้น',
 'ขายแล้ว 18.8พัน ชิ้น',
 'ขายแล้ว 3.3พัน ชิ้น',
 'ขายแล้ว 3.7พัน ชิ้น',
 'ขายแล้ว 289 ชิ้น',
 'ขายแล้ว 557 ชิ้น',
 'ขายแล้ว 345 ชิ้น',
 'ขายแล้ว 3.7พัน ชิ้น',
 'ขายแล้ว 2พัน ชิ้น',
 'ขายแล้ว 738 ชิ้น',
 'ขายแล้ว 61 ชิ้น',
 'ขายแล้ว 848 ชิ้น',
 'ขายแล้ว 21.7พัน ชิ้น',
 'ขายแล้ว 37.4พัน ชิ้น',
 'ขายแล้ว 847 ชิ้น',
 'ขายแล้ว 1.8พัน ชิ้น',
 'ขายแล้ว 929 ชิ้น',
 'ขายแล้ว 4.1พัน ชิ้น',
 'ขายแล้ว 3พัน ชิ้น',

In [24]:
all_province

['จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรปราการ',
 'จังหวัดปทุมธานี',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรสาคร',
 'จังหวัดปทุมธานี',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดปทุมธานี',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดกรุงเทพมหานคร',
 'จังหวัดสมุทรปราการ',
 'จังหวัดสมุทรปราการ',
 'จังหว

In [25]:
#รวม data เข้าด้วยกันไว้ใน list

allData_list = list(zip(all_products, all_price, all_soldAmount, all_province))

In [26]:
#เปลี่ยน data ที่อยู่ใน list เป็น data frame ที่มี head column คือ Products, Price, Sold Amount, Province

working_table_data = pd.DataFrame(allData_list, columns = ['Products', 'Price', 'Sold Amount', 'Province'])

In [27]:
#เช็คว่า data มีการรับค่ามาซ้ำกันไหม ถ้าซ้ำก็ลบออกให้เหลือเพียงตัวเดียว
working_table_data.drop_duplicates()

,Products,Price,Sold Amount,Province
0,ALOVE【มี 3 สี】80/100/120cmโต๊ะทํางาน ราคาถูก ...,฿124 - ฿454,ขายแล้ว 16.9พัน ชิ้น,จังหวัดสมุทรปราการ
1,【มี 3 สี】มั่นคง โต๊ะเรียนขาเหล็กหนา 80/100/120cm,฿124 - ฿454,ขายแล้ว 23พัน ชิ้น,จังหวัดสมุทรปราการ
2,โต๊ะวางโน๊ตบุ๊ค สำหรับเด็ก นักเรียน นักศึกษา ว...,฿85 - ฿125,ขายแล้ว 2.2พัน ชิ้น,จังหวัดกรุงเทพมหานคร
3,【Cles】🔥ราคาโปร‼️ โต๊ะคอม 💻โต๊ะวางโน้ตบุ๊ค โต๊ะ...,฿109,ขายแล้ว 78 ชิ้น,จังหวัดกรุงเทพมหานคร
4,[ขาโต๊ะหนา]โต๊ะทํางานสีขาว80/100/120/140CM นัก...,"฿494 - ฿1,039",ขายแล้ว 1.5พัน ชิ้น,จังหวัดสมุทรปราการ
...,...,...,...,...
957,โต๊ะทำงาน พร้อมชั้นวางหนังสือด้านข้าง ชั้นวางข...,฿999,ขายแล้ว 62 ชิ้น,จังหวัดปทุมธานี
958,โต๊ะคอมพิวเตอร์ โต๊ะทำงาน TB-121,"฿2,746",ขายแล้ว 261 ชิ้น,จังหวัดกรุงเทพมหานคร
959,โต๊ะคอมพิวเตอร์ โต๊ะวาง Computer CD-805 Civil ...,"฿1,087 - ฿1,590",ขายแล้ว 11 ชิ้น,จังหวัดกรุงเทพมหานคร
981,(P-119) ปากกาสไตล์MUJI หมึกเยอะ !! ขนาด 0.5mm ...,฿1,ขายแล้ว 998พัน ชิ้น,จังหวัดสมุทรปราการ


In [ ]:
#save เป็นไฟล์ excel

working_table_data.to_excel(r'C:\Users\your username\Desktop\shopee\shopee_scraping_dropDup.xlsx')